# Framing The Correct Problem

Before we proceed with any coding or modeling, it is important to specify the exact nature of what we are trying to accomplish.

The ultimate goal of this project is create a recommender system based on <a href='https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations'> H&M's online sales data </a> from 2018 to 2020. The data set comprises of:
1. A log of every online transaction by user id and product id over the 2 year span. Contains 31,788,324 transactions in total.
2. Meta data on each product by product id, including apparel type, department, etc.
3. An image corresponding to products by product id. Note that some images may be corrupted, unintelligible, or straight-up missing.

Firstly, we must decide on what exactly our recommender system is trying to predict. At its core, a recommender system takes pre-recorded interactions between users and items and tries to predict future/unknown interactions between specific users with specific items. 

Notice that we have transaction data but not rating data. This means each user's interaction with the product is binary variable: 0 for not purchased, 1 for purchased. The binary nature of this data leads to our first interesting theoretical problem.

**Question)** If a user-item interaction value is 0 (not purchased) is it because:
- the user does not know of the item?
- the user did not like the item?

In other words, one of the major theoretical challenges presented by this data is the ambiguity in the *reason why a user did not buy an item*. Is it because they just don't know about it? Or is it because they didn't actually like it?

Notice this same issue does not appear in more classical recommender models such as movie recommendations. In classical data sets (like Movie Lens) there is a very clear demarkation between "did not see" vs "did not like". A rating of 5 means the user saw and liked the movie, a rating of 1 means the user saw and did not like the movie, and a missing value is interpreted as the user did not see the movie.


---
<br>

So what exactly do we want our recommender system to do. A naive answer might be the following:

**Objective 1)** Given a user's past purchases, predict how likely a user would purchase a given item, if they were shown that item.

However, if we ponder for a second how we would achieve this, we quickly realize a fundamental issue: in order for a model to correctly predict a probability, it needs to have access to negative cases. In our data set, we only have access to positive cases, which leads to an issue: the model can just cheat and guess "users like everything and will buy anything with probability 1". We have no way of penalizing such a guess because we don't have any clear examples of users NOT liking a product.

One way to get around this roadblock is to artificially generate negative examples and this will require us to make assumptions on the nature of the data. The key issue is that we cannot distinguish between true negative interactions (users did not like the item) vs missing interactions (users did not see the item). So what if we made the simplifying assumption that all users must have seen a certain collection of items.

More specifically, let's fix a time period, say Jan 2019 to Feb 2019. Let I(1), I(2), ..., I(10) denote the top 10 best sellers over this time period. Because these items were so popular, it seems reasonable to assume many people saw these items on the webpage during this time period. What if we made the following simplifying assumption?

**Assumption)** all users on the webpage saw items I(1), ... , I(10) and missing interactions indicate the users saw but did not like the item enough to buy it.

This now gives us a concrete set of negative examples which we can then contrast with positive examples (users buying an item) and we can then use this predict what a user will buy in the next time period.

---

<br>

Of course, framing the problem as we did above comes with the caveat that we are making a rather heavy assumption on the visibility of products by popularity. Let us explore another objective that does not require such a heavy assumption. 

Instead of treating each user as a separate entity, what if we pooled all users together into a single entity, i.e. a "general public" or "consumer" entity. Instead of trying to recommend individual items to individual users, what if we instead tried to learn the trend of item sales over time. The idea is to build a model which can forecast the sales number of an individual item based entirely on the picture of the item.

**Objective 2)** Use the image of each clothing item to forecast sales numbers over time.

The idea is that in learning to make this forecast, the model will have to implicilty learn about seasonal trends and fashion/style trends across time. E.g. the model will have to recognize summer vs winter clothing, stylistic vs "ugly" clothing, etc. Once we know if an item will be a "hot seller" we can then make recommendations to all users. If the model can forecast sales numbers accurately, we also have the secondary benefit of being able to resolve supply-chain and production issues.

This objective is much more well-founded in the sense that we do not need to make assumptions about the data. We also cut down on the "size" of the data set because we no longer need to distinguish individual users. Of course, this comes at the cost of no longer being able to make personalized recommendations.

In [2]:
from torchvision.io import read_image

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import support_victor_machine as support

In [5]:
transactions = pd.read_csv('../data/transactions_train.csv')

articles = pd.read_csv('../data/articles.csv')

In [26]:
transactions.shape

(31788324, 5)

In [22]:
len(articles['article_id'])

105542

In [19]:
has_img = []

for i in articles['article_id']:
    file = '0'+str(i)
    folder = file[0:3]
    try:
        read_image('../resized_images/'+folder+'/'+file+'.jpg')
        has_img.append(i)
    except:
        pass
    

In [21]:
len(has_img)

105100

In [24]:
transactions_with_img = transactions[ transactions['article_id'].isin(has_img) ]

In [25]:
transactions_with_img.shape

(31651678, 5)

In [28]:
transactions_with_img.to_csv('../data/transactions_cleaned.csv')